# Better ML Engineering with ML Metadata



## Learning Objectives

1. Download the dataset
2. Create an InteractiveContext
3. Construct the TFX Pipeline
4. Query the MLMD Database



## Introduction 



Assume a scenario where you set up a production ML pipeline to classify penguins. The pipeline ingests your training data, trains and evaluates a model, and pushes it to production.

However, when you later try using this model with a larger dataset that contains different kinds of penguins, you observe that your model does not behave as expected and starts classifying the species incorrectly.

At this point, you are interested in knowing:

* What is the most efficient way to debug the model when the only available artifact is the model in production?
* Which training dataset was used to train the model?
* Which training run led to this erroneous model?
* Where are the model evaluation results?
* Where to begin debugging?

[ML Metadata (MLMD)](https://github.com/google/ml-metadata) is a library that leverages the metadata associated with ML models to help you answer these questions and more. A helpful analogy is to think of this metadata as the equivalent of logging in software development. MLMD enables you to reliably track the artifacts and lineage associated with the various components of your ML pipeline.

In this notebook, you set up a TFX Pipeline to create a model that classifies penguins into three species based on the body mass and the length and depth of their culmens, and the length of their flippers. You then use MLMD to track the lineage of pipeline components.

Each learning objective will correspond to a __#TODO__ in the [student lab notebook](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/production_ml/labs/mlmd_tutorial.ipynb) -- try to complete that notebook first before reviewing this solution notebook. 


## Setup

First, we install and import the necessary packages, set up paths, and download data.

### Upgrade Pip



In [1]:
!pip install --upgrade pip

     |████████████████████████████████| 1.5MB 7.5MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


### Install and import TFX

In [2]:
!pip install -q -U tfx

     |████████████████████████████████| 2.3 MB 8.7 MB/s 
     |████████████████████████████████| 2.9 MB 37.5 MB/s 
     |████████████████████████████████| 9.0 MB 33.3 MB/s 
     |████████████████████████████████| 103 kB 26.8 MB/s 
     |████████████████████████████████| 1.7 MB 43.3 MB/s 
     |████████████████████████████████| 1.4 MB 40.4 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 89 kB 5.9 MB/s 
     |████████████████████████████████| 147 kB 55.7 MB/s 
     |████████████████████████████████| 400 kB 53.9 MB/s 
     |████████████████████████████████| 1.5 MB 67.6 MB/s 
     |████████████████████████████████| 1.2 MB 54.5 MB/s 
     |████████████████████████████████| 17.7 MB 82 kB/s 
     |████████████████████████████████| 2.2 MB 43.5 MB/s 
     |████████████████████████████████| 829 kB 49.0 MB/s 
     |████████████████████████████████| 151 kB 52.2 MB/s 
     |████████████████████████████████| 2.2 MB 44.8 MB/s 
     |██████████████

Please ignore the incompatibility error and warnings.

You must restart the kernel after installing TFX. Select **Kernel > Restart kernel > Restart** from the menu.

Do not proceed with the rest of this notebook without restarting the kernel.

### Import other libraries

In [2]:
import os
import tempfile
import urllib
import pandas as pd

Import [TFX component](https://tensorflow.google.cn/tfx/tutorials/tfx/components_keras) classes.

In [3]:
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import external_input

from tensorflow_metadata.proto.v0 import anomalies_pb2
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_metadata.proto.v0 import statistics_pb2

import tensorflow_model_analysis as tfma

from tfx.components import ResolverNode
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

Import the MLMD library.

In [4]:
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2

## Download the dataset

We use the [Palmer Penguins dataset](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) which can be found on [Github](https://github.com/allisonhorst/palmerpenguins). We processed the dataset by leaving out any incomplete records, and drops `island` and `sex` columns, and converted labels to `int32`. The dataset contains 334 records of the body mass and the length and depth of penguins' culmens, and the length of their flippers. You use this data to classify penguins into one of three species.

In [5]:
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/penguins_processed.csv'
_data_root = tempfile.mkdtemp(prefix='tfx-data')
# TODO
# Join various path components
_data_filepath = os.path.join(_data_root, "penguins_processed.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tfx-datarn01z3vz/penguins_processed.csv',
 <http.client.HTTPMessage at 0x7f8cafe17b50>)

## Create an InteractiveContext

To run TFX components interactively in this notebook, create an `InteractiveContext`. The `InteractiveContext` uses a temporary directory with an ephemeral MLMD database instance.

In general, it is a good practice to group similar pipeline runs under a `Context`.

In [6]:
# TODO
interactive_context = InteractiveContext()

## Construct the TFX Pipeline

A TFX pipeline consists of several components that perform different aspects of the ML workflow. In this notebook, you create and run the `ExampleGen`, `StatisticsGen`, `SchemaGen`, and `Trainer` components and use the `Evaluator` and `Pusher` component to evaluate and push the trained model. 

Refer to the [components tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras) for more information on TFX pipeline components.

Note: Constructing a TFX Pipeline by setting up the individual components involves a lot of boilerplate code. For the purpose of this notebook, it is alright if you do not fully understand every line of code in the pipeline setup. 

### Instantiate and run the ExampleGen Component

In [7]:
input_data = external_input(_data_root)
# TODO
example_gen = CsvExampleGen(input=input_data)
interactive_context.run(example_gen)

Instructions for updating:
external_input is deprecated, directly pass the uri to ExampleGen.


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-04-19T10_22_06.380156-emsem0yo/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:25648,xor_checksum:1618827726,sum_checksum:1618827726"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Instantiate and run the StatisticsGen Component

In [8]:
# TODO
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/tmp/tfx-interactive-2021-04-19T10_22_06.380156-emsem0yo/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Instantiate and run the SchemaGen Component

In [9]:
# TODO
infer_schema = SchemaGen(
    statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
interactive_context.run(infer_schema)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/tmp/tfx-interactive-2021-04-19T10_22_06.380156-emsem0yo/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Instantiate and run the Trainer Component








In [10]:
# Define the module file for the Trainer component
trainer_module_file = 'penguin_trainer.py'

In [11]:
%%writefile {trainer_module_file}

# Define the training algorithm for the Trainer module file
import os
from typing import List, Text

import tensorflow as tf
from tensorflow import keras

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import FnArgs

# Features used for classification - culmen length and depth, flipper length,
# body mass, and species.

_FEATURES = {
    'culmen_length_mm': tf.io.FixedLenFeature([], dtype=tf.float32),
    'culmen_depth_mm': tf.io.FixedLenFeature([], dtype=tf.float32),
    'flipper_length_mm': tf.io.FixedLenFeature([], dtype=tf.float32),
    'body_mass_g': tf.io.FixedLenFeature([], dtype=tf.float32),
    'species': tf.io.FixedLenFeature([], dtype=tf.int64)
}

_LABEL_KEY = 'species'

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]


def _gzip_reader_fn(filenames):
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def _input_fn(file_pattern: List[Text], batch_size: int = 200):
  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=_FEATURES,
      reader=_gzip_reader_fn,
      label_key=_LABEL_KEY)

  return dataset


def _build_keras_model():
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  d = keras.layers.Dense(8, activation='relu')(d)
  d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3, activation='softmax')(d)
  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer='adam',
      loss='sparse_categorical_crossentropy',
      metrics=[keras.metrics.SparseCategoricalAccuracy()])
  return model


def run_fn(fn_args: TrainerFnArgs):
  train_dataset = _input_fn(fn_args.train_files, batch_size=10)
  eval_dataset = _input_fn(fn_args.eval_files, batch_size=10)
  model = _build_keras_model()
  model.fit(
      train_dataset,
      epochs=int(fn_args.train_steps / 20),
      steps_per_epoch=20,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)
  model.save(fn_args.serving_model_dir, save_format='tf')

Writing penguin_trainer.py


Run the `Trainer` component.

In [12]:
trainer = Trainer(
    module_file=os.path.abspath(trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=example_gen.outputs['examples'],
    schema=infer_schema.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=50))
interactive_context.run(trainer)

Epoch 1/5
20/20 [==============================] - 2s 41ms/step - loss: 1.0736 - sparse_categorical_accuracy: 0.4434 - val_loss: 1.0587 - val_sparse_categorical_accuracy: 0.3300
Epoch 2/5
20/20 [==============================] - 0s 9ms/step - loss: 1.0446 - sparse_categorical_accuracy: 0.3951 - val_loss: 1.0321 - val_sparse_categorical_accuracy: 0.3800
Epoch 3/5
20/20 [==============================] - 0s 8ms/step - loss: 1.0075 - sparse_categorical_accuracy: 0.4740 - val_loss: 1.0068 - val_sparse_categorical_accuracy: 0.5600
Epoch 4/5
20/20 [==============================] - 0s 9ms/step - loss: 0.9766 - sparse_categorical_accuracy: 0.6205 - val_loss: 0.9821 - val_sparse_categorical_accuracy: 0.6400
Epoch 5/5
20/20 [==============================] - 0s 9ms/step - loss: 0.9722 - sparse_categorical_accuracy: 0.5671 - val_loss: 0.9564 - val_sparse_categorical_accuracy: 0.6700
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-04-19T10_22_06.380156-emsem0yo/Trainer/model/4/Format

INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-04-19T10_22_06.380156-emsem0yo/Trainer/model/4/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 4
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/tmp/tfx-interactive-2021-04-19T10_22_06.380156-emsem0yo/Trainer/model/4"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 5
        type_id: 12
        uri: "/tmp/tfx-interactive-2021-04-19T10_22_06.380156-emsem0yo/Trainer/model_run/4"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 12
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Evaluate and push the model

Use the `Evaluator` component to evaluate and 'bless' the model before using the `Pusher` component to push the model to a serving directory.

In [13]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(),
                                  'serving_model/penguins_classification')

In [14]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='species', signature_name='serving_default')
    ],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(
                class_name='SparseCategoricalAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.6})))
        ])
    ],
    slicing_specs=[tfma.SlicingSpec()])

In [15]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    schema=infer_schema.outputs['schema'],
    eval_config=eval_config)
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 5
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "/tmp/tfx-interactive-2021-04-19T10_22_06.380156-emsem0yo/Evaluator/evaluation/5"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 7
        type_id: 15
        uri: "/tmp/tfx-interactive-2021-04-19T10_22_06.380156-emsem0yo/Evaluator/blessing/5"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/tmp/tfx-interactive-2021-04-19T10_22_06.380156-emsem0yo/Trainer/model/4"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 4
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [16]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 6
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 8
        type_id: 17
        uri: "/tmp/tfx-interactive-2021-04-19T10_22_06.380156-emsem0yo/Pusher/pushed_model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "/tmp/tmp1clqtrsl/serving_model/penguins_classification/1618827742"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1618827742"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

Running the TFX pipeline populates the MLMD Database. In the next section, you use the MLMD API to query this database for metadata information. 

## Query the MLMD Database

The MLMD database stores three types of metadata: 

*    Metadata about the pipeline and lineage information associated with the pipeline components
*    Metadata about artifacts that were generated during the pipeline run
*    Metadata about the executions of the pipeline

A typical production environment pipeline serves multiple models as new data arrives. When you encounter erroneous results in served models, you can query the MLMD database to isolate the erroneous models. You can then trace the lineage of the pipeline components that correspond to these models to debug your models

Set up the metadata (MD) store with the `InteractiveContext` defined previously to query the MLMD database.

In [17]:
connection_config = interactive_context.metadata_connection_config
store = mlmd.MetadataStore(connection_config)

# All TFX artifacts are stored in the base directory
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

Create some helper functions to view the data from the MD store.

In [18]:
def display_types(types):
  # Helper function to render dataframes for the artifact and execution types
  table = {'id': [], 'name': []}
  for a_type in types:
    table['id'].append(a_type.id)
    table['name'].append(a_type.name)
  return pd.DataFrame(data=table)

In [19]:
def display_artifacts(store, artifacts):
  # Helper function to render dataframes for the input artifacts
  table = {'artifact id': [], 'type': [], 'uri': []}
  for a in artifacts:
    table['artifact id'].append(a.id)
    artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
    table['type'].append(artifact_type.name)
    table['uri'].append(a.uri.replace(base_dir, './'))
  return pd.DataFrame(data=table)

In [20]:
def display_properties(store, node):
  # Helper function to render dataframes for artifact and execution properties
  table = {'property': [], 'value': []}
  for k, v in node.properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  for k, v in node.custom_properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  return pd.DataFrame(data=table)

First, query the MD store for a list of all its stored `ArtifactTypes`.

In [21]:
display_types(store.get_artifact_types())

,id,name
0,5,Examples
1,7,ExampleStatistics
2,9,Schema
3,11,Model
4,12,ModelRun
5,14,ModelEvaluation
6,15,ModelBlessing
7,17,PushedModel


Next, query all `PushedModel` artifacts.

In [22]:
pushed_models = store.get_artifacts_by_type("PushedModel")
display_artifacts(store, pushed_models)

,artifact id,type,uri
0,8,PushedModel,./Pusher/pushed_model/6


Query the MD store for the latest pushed model. This notebook has only one pushed model. 

In [23]:
pushed_model = pushed_models[-1]
display_properties(store, pushed_model)

,property,value
0,state,published
1,producer_component,Pusher
2,pushed_destination,/tmp/tmp1clqtrsl/serving_model/penguins_classi...
3,tfx_version,0.29.0
4,pushed,1
5,name,pushed_model
6,pushed_version,1618827742


One of the first steps in debugging a pushed model is to look at which trained model is pushed and to see which training data is used to train that model.  

MLMD provides traversal APIs to walk through the provenance graph, which you can use to analyze the model provenance. 

In [24]:
def get_one_hop_parent_artifacts(store, artifacts):
  # Get a list of artifacts within a 1-hop of the artifacts of interest
  artifact_ids = [artifact.id for artifact in artifacts]
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids(artifact_ids)
      if event.type == metadata_store_pb2.Event.OUTPUT)
  artifacts_ids = set(
      event.artifact_id
      for event in store.get_events_by_execution_ids(executions_ids)
      if event.type == metadata_store_pb2.Event.INPUT) 
  return [artifact for artifact in store.get_artifacts_by_id(artifacts_ids)]

Query the parent artifacts for the pushed model.

In [25]:
# TODO
parent_artifacts = get_one_hop_parent_artifacts(store, [pushed_model])
display_artifacts(store, parent_artifacts)

,artifact id,type,uri
0,4,Model,./Trainer/model/4
1,7,ModelBlessing,./Evaluator/blessing/5


Query the properties for the model.

In [26]:
exported_model = parent_artifacts[0]
display_properties(store, exported_model)

,property,value
0,state,published
1,producer_component,Trainer
2,name,model
3,tfx_version,0.29.0


Query the upstream artifacts for the model.

In [27]:
model_parents = get_one_hop_parent_artifacts(store, [exported_model])
display_artifacts(store, model_parents)

,artifact id,type,uri
0,1,Examples,./CsvExampleGen/examples/1
1,3,Schema,./SchemaGen/schema/3


Get the training data the model trained with.

In [28]:
used_data = model_parents[0]
display_properties(store, used_data)

,property,value
0,split_names,"[""train"", ""eval""]"
1,payload_format,FORMAT_TF_EXAMPLE
2,tfx_version,0.29.0
3,input_fingerprint,"split:single_split,num_files:1,total_bytes:256..."
4,state,published
5,span,0


Now that you have the training data that the model trained with, query the database again to find the training step (execution). Query the MD store for a list of the registered execution types.

In [29]:
display_types(store.get_execution_types())

,id,name
0,3,tfx.components.example_gen.csv_example_gen.com...
1,6,tfx.components.statistics_gen.component.Statis...
2,8,tfx.components.schema_gen.component.SchemaGen
3,10,tfx.components.trainer.component.Trainer
4,13,tfx.components.evaluator.component.Evaluator
5,16,tfx.components.pusher.component.Pusher


The training step is the `ExecutionType` named `tfx.components.trainer.component.Trainer`. Traverse the MD store to get the trainer run that corresponds to the pushed model.

In [30]:
def find_producer_execution(store, artifact):
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids([artifact.id])
      if event.type == metadata_store_pb2.Event.OUTPUT)
  return store.get_executions_by_id(executions_ids)[0]

# TODO
trainer = find_producer_execution(store, exported_model)
display_properties(store, trainer)

,property,value
0,component_id,Trainer
1,pipeline_root,/tmp/tfx-interactive-2021-04-19T10_22_06.38015...
2,module_file,/content/penguin_trainer.py
3,checksum_md5,bfcbe87349c87f86751e959786dde8f4
4,train_args,"{\n ""num_steps"": 100\n}"
5,custom_config,null
6,run_fn,None
7,state,complete
8,pipeline_name,interactive-2021-04-19T10_22_06.380156
9,trainer_fn,None


## Summary

In this tutorial, you learned about how you can leverage MLMD to trace the lineage of your TFX pipeline components and resolve issues.

To learn more about how to use MLMD, check out these additional resources:

* [MLMD API documentation](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd)
* [MLMD guide](https://www.tensorflow.org/tfx/guide/mlmd)